In [6]:
pip install -U numpy scipy scikit-learn pandas tweet_preprocessor


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import sklearn
import numpy as np
import pandas as pd

In [8]:
#import data 
tweets = pd.read_csv("/Users/denisetan/Downloads/Hate_speech-main/dataset_hate.csv")

In [9]:
y = (tweets.label == 0) 
x = (tweets.label == 1)
x_train, x_test , y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [10]:
tweets.head(10)

,Unnamed: 0,id,label,tweet
0,0,1,0,@user when a father is dysfunctional and is s...
1,1,2,0,@user @user thanks for #lyft credit i can't us...
2,2,3,0,bihday your majesty
3,3,4,0,#model i love u take with u all the time in ...
4,4,5,0,factsguide: society now #motivation
5,5,6,0,[2/2] huge fan fare and big talking before the...
6,6,7,0,@user camping tomorrow @user @user @user @use...
7,7,8,0,the next school year is the year for exams.ð...
8,8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,9,10,0,@user @user welcome here ! i'm it's so #gr...


In [11]:
tweets.tail()

,Unnamed: 0,id,label,tweet
31957,31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31958,31959,0,to see nina turner on the airwaves trying to...
31959,31959,31960,0,listening to sad songs on a monday morning otw...
31960,31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31961,31962,0,thank you @user for you follow


In [12]:
# non-hate tweets 
sum(tweets["label"] == 0)

29720

In [13]:
# hate tweets 
sum(tweets["label"] == 1)

2242

In [14]:
# check if there are any missing values
tweets.isnull().sum()

Unnamed: 0    0
id            0
label         0
tweet         0
dtype: int64

In [15]:
# Data cleaning 

In [16]:
# remove special characters 
import re 

# set up punctuations to be replaced 
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [17]:
import preprocessor as p 

In [18]:
def clean_tweets(df):
    tempArr = []
    for line in df: 
        tmpL = p.clean(line)
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) #convert all tweets to lower case
        tmpL = REPLACE_WITH_SPACE.sub("", tmpL)
        tempArr.append(tmpL)
    return tempArr

In [19]:
# append cleaned tweets to data 
cleaned_tweets = clean_tweets(tweets["tweet"])
cleaned_tweets = pd.DataFrame(cleaned_tweets)

In [20]:
cleaned_tweets

,0
0,when a father is dysfunctional and is so selfi...
1,thanks for credit i cant use cause they dont o...
2,bihday your majesty
3,i love u take with u all the time in ur
4,factsguide society now
...,...
31957,ate isz that youuu
31958,to see nina turner on the airwaves trying to w...
31959,listening to sad songs on a monday morning otw...
31960,vandalised in in condemns act


In [21]:
# split clean data 
from sklearn.model_selection import train_test_split

x = cleaned_tweets[0]
y = tweets["label"]
x_train, x_test , y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)


In [23]:
# vectorize tweets using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, stop_words='english')
vectorizer.fit(list(x_train) + list(x_test))
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [24]:
# apply SVC
from sklearn import svm 
svm = svm.SVC(kernel = 'linear', probability = True)
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)
y_pred_svm = svm.predict(x_test_vec)

In [28]:
from sklearn.metrics import accuracy_score, precision_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')
print("Precision score for SVC is: ", precision_score(y_test, y_pred_svm) * 100, '%') # precision score is low

Accuracy score for SVC is:  95.27608321601751 %
Precision score for SVC is:  68.27586206896552 %
